# VactionPy

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df= pd.read_csv("output_data/cities.csv")
# Display Samople data
city_data_df.head()
# city_data_df.dtypes

,City_Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,-9.75,85,98,5.52,RU,1699737536
1,1,edinburgh of the seven seas,-37.0676,-12.3116,14.10,86,70,7.07,SH,1699737536
2,2,fort mcmurray,56.7268,-111.3810,0.77,86,40,5.14,CA,1699737353
3,3,ribeira grande,38.5167,-28.7000,20.22,94,75,7.15,PT,1699737537
4,4,port-aux-francais,-49.3500,70.2167,3.00,68,10,7.33,TF,1699737537


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng",
                                        "Lat", 
                                        geo = True,
                                        tiles = "OSM",
                                        frame_width =700,
                                        frame_height= 500,
                                        size ="Humidity",
                                        scale =.75,
                                        color ="City"
                                       )


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21

# Wind speed less than 4.5 m/s

# Zero cloudiness


narrow_down_df = city_data_df.loc[(city_data_df['Max Temp']>21)
                                    & (city_data_df['Max Temp']<30)
                                    & (city_data_df['Cloudiness']==0) 
                                    & (city_data_df['Wind Speed']<4.5)
                                  ]
                                    
# Drop any rows with null values
narrow_down_df = narrow_down_df.dropna()

# Display sample data
narrow_down_df
# narrow_down_df.dtypes

,City_Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
163,163,kish,26.5578,54.0194,26.15,65,0,2.06,IR,1699737560
237,237,isla vista,34.4133,-119.8610,28.89,24,0,2.06,US,1699737571
299,299,waddan,29.1614,16.1390,22.36,24,0,1.78,LY,1699737582
304,304,pamandzi,-12.7967,45.2794,26.88,83,0,1.54,YT,1699737583
316,316,pacific grove,36.6177,-121.9166,23.47,49,0,3.09,US,1699737374
323,323,siwa oasis,29.2041,25.5195,21.78,49,0,0.86,EG,1699737585
333,333,rocha,-34.4833,-54.3333,21.72,68,0,3.83,UY,1699737511
442,442,toliara,-23.3500,43.6667,23.63,56,0,4.49,MG,1699737608
443,443,salinas,36.6777,-121.6555,26.66,40,0,3.09,US,1699737448
466,466,awjilah,29.1081,21.2869,21.43,42,0,2.61,LY,1699737612


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# hotel_df = narrow_down_df.copy("City","Country","Lat","Lng","Humidity")
hotel_df = narrow_down_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
163,kish,IR,26.5578,54.0194,65,
237,isla vista,US,34.4133,-119.8610,24,
299,waddan,LY,29.1614,16.1390,24,
304,pamandzi,YT,-12.7967,45.2794,83,
316,pacific grove,US,36.6177,-121.9166,49,
323,siwa oasis,EG,29.2041,25.5195,49,
333,rocha,UY,-34.4833,-54.3333,68,
442,toliara,MG,-23.3500,43.6667,56,
443,salinas,US,36.6777,-121.6555,40,
466,awjilah,LY,29.1081,21.2869,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
# conditions = "dogs.yes"
limit = 20

params = {
     "categories":categories,
#      "conditions":conditions,
     "apiKey":geoapify_key,
     "limit" : limit
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"] 
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    
    name_address = response.json()
    
#     print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kish - nearest hotel: هتل فارابی
isla vista - nearest hotel: The Club & Guest House
waddan - nearest hotel: No hotel found
pamandzi - nearest hotel: Gite Le Madina
pacific grove - nearest hotel: Pacific Grove Inn
siwa oasis - nearest hotel: فندق الكيلانى
rocha - nearest hotel: Hotel Municipal
toliara - nearest hotel: Ambary
salinas - nearest hotel: The Traveler's Hotel
awjilah - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
tiangua - nearest hotel: Pousada


,City,Country,Lat,Lng,Humidity,Hotel Name
163,kish,IR,26.5578,54.0194,65,هتل فارابی
237,isla vista,US,34.4133,-119.8610,24,The Club & Guest House
299,waddan,LY,29.1614,16.1390,24,No hotel found
304,pamandzi,YT,-12.7967,45.2794,83,Gite Le Madina
316,pacific grove,US,36.6177,-121.9166,49,Pacific Grove Inn
323,siwa oasis,EG,29.2041,25.5195,49,فندق الكيلانى
333,rocha,UY,-34.4833,-54.3333,68,Hotel Municipal
442,toliara,MG,-23.3500,43.6667,56,Ambary
443,salinas,US,36.6777,-121.6555,40,The Traveler's Hotel
466,awjilah,LY,29.1081,21.2869,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
                                    "Lng",
                                    "Lat", 
                                     geo = True,
                                     tiles = "OSM",
                                     frame_width =800,
                                     frame_height= 600,
                                     size ="Humidity",
                                     scale =.75,
                                     color = "City",
                                     hover_cols =["Country","Hotel Name"]
                                    )


# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)